In [79]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import gc
from pathlib import Path
import os

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from IPython.display import FileLink

import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

import PIL
from PIL import Image as PIL_IMG

import seaborn as sns

In [ ]:
print("Fastai version: ", fastai.__version__)
print("PIL version: ", PIL.__version__)

In [ ]:
DATA_DIR = Path('../input')
IMGS_PATH = Path('../input/images/images/')

In [ ]:
[f for f in DATA_DIR.iterdir()]

In [ ]:
train_df = pd.read_csv(DATA_DIR/'train.csv')
test_df = pd.read_csv(DATA_DIR/'test.csv')

In [ ]:
label_mapper = {0: 'Cargo', 
                1: 'Military', 
                2: 'Carrier', 
                3: 'Cruise', 
                4: 'Tankers'}

In [94]:
label_mapper_c2i = {v:i for i, v in label_mapper.items()}

In [ ]:
train_df['category'] = train_df['category'].map(label_mapper)

In [ ]:
train_df.head()

In [ ]:
# functions to get image statistics

def getSize(filename):
    """
    param filename : Image file name in string format
    returns : size of an image in Kilo Bytes
    """
    filename = IMGS_PATH/filename
    st = os.stat(filename)
    return st.st_size / 1024.0

def getDimensions(filename):
    """
    param filename : Image file name in string format
    returns : Image size (width, height) tuple
    """
    filename = IMGS_PATH/filename
    img_size = PIL_IMG.open(filename).size
    return img_size 

In [ ]:
train_img_sizes = train_df[~train_df['valid']]['image'].apply(getSize)
valid_img_sizes = train_df[train_df['valid']]['image'].apply(getSize)
test_img_sizes = test_df['image'].apply(getSize)

In [ ]:
fig ,ax = plt.subplots(ncols=3, nrows=1, figsize=(15,4))
for i, d in enumerate(zip([train_img_sizes, valid_img_sizes, test_img_sizes], ['train', 'validation', 'test'])):
    ax_s = sns.distplot(d[0], ax=ax[i])
    ax_s.set(title=d[1] + " file size distribution")
    
plt.tight_layout()
plt.show()

In [ ]:
train_img_dims = train_df[~train_df['valid']]['image'].apply(getDimensions)
valid_img_dims = train_df[train_df['valid']]['image'].apply(getDimensions)
test_img_dims = test_df['image'].apply(getDimensions)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=3, figsize=(12,12))
for i, d in enumerate(zip([train_img_dims, valid_img_dims, test_img_dims], ['train', 'validation', 'test'])):
    widths = d[0].apply(lambda x: x[0])
    heights = d[0].apply(lambda x: x[1])
    ax_w = sns.distplot(widths, ax=ax[i][0])
    ax_w.set(title=d[1] + " width distribution")
    ax_h = sns.distplot(heights, ax=ax[i][1])
    ax_h.set(title=d[1] + " height distribution")

plt.tight_layout()
plt.show()

We have to change Width and height.

In [ ]:
def get_ex(path): return open_image(path)

def plots_f(path, tfms, rows, cols, width, height, **kwargs):
    [get_ex(path).apply_tfms(tfms[0], **kwargs).show(ax=ax) for i,ax in enumerate(plt.subplots(rows,cols,figsize=(width,height))[1].flatten())]

In [ ]:
tfms = get_transforms(xtra_tfms=[jitter(magnitude=0.005, p=.25)])

In [ ]:
tfms

In [ ]:
open_image(IMGS_PATH/'2816925.jpg')

In [ ]:
plots_f(IMGS_PATH/'2816925.jpg', tfms, 2, 4, 12, 6, size=224)

In [ ]:
train_il = ImageList.from_df(df=train_df, cols='image', path=IMGS_PATH)
test_il = ImageList.from_df(df=test_df, cols='image', path=IMGS_PATH)

In [ ]:
np.random.seed(0)
src1 = (train_il
        .split_from_df(col='valid')
        .label_from_df(cols='category')
        .add_test(test_il))

In [ ]:
tfms1 = get_transforms()     
data1 = (src1
        .transform(tfms1, size=224)
        .databunch(path='.', bs=128, num_workers=0)
        .normalize(imagenet_stats))

In [ ]:
data1

In [ ]:
data1.show_batch()

In [ ]:
learn1 = cnn_learner(data1, models.resnet50, metrics=[accuracy, FBeta(average='weighted')])

In [ ]:
learn1.lr_find()

In [ ]:
learn1.recorder.plot()

In [ ]:
rn50_callbacks = [CSVLogger(learn=learn1, filename='rn50_history'), ShowGraph(learn=learn1)]

In [ ]:
lr = (1e-2)/2
learn1.fit_one_cycle(5, slice(lr), callbacks=rn50_callbacks)

In [ ]:
learn1.save('rn50-stage1')

In [ ]:
learn1.unfreeze()
learn1.lr_find()
learn1.recorder.plot()

In [ ]:
rn50_stage2_callbacks = [CSVLogger(learn=learn1, filename='rn50_stage2_history'),ShowGraph(learn=learn1)]

In [ ]:
learn1.fit_one_cycle(5, slice(1e-5, lr/5), callbacks=rn50_stage2_callbacks)

In [ ]:
learn1.save('rn50-stage2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn1)
interp.plot_top_losses(9)

In [ ]:
interp.plot_confusion_matrix()

As we can see that our algorithm is more confused about : Cargo and Tankers

In [ ]:
from fastai.callbacks.hooks import *
import cv2
import matplotlib.pyplot as plt
import random

# hook into forward pass
def hooked_backward(m, oneBatch, cat):
    # we hook into the convolutional part = m[0] of the model
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(oneBatch)
            preds[0,int(cat)].backward()
    return hook_a,hook_g

# We can create a utility function for getting a validation image with an activation map
def getHeatmap(val_index, learner, imgDataBunch):
    """Returns the validation set image and the activation map"""
    # this gets the model
    m = learner.model.eval()
    tensorImg,cl = imgDataBunch.valid_ds[val_index]
    # create a batch from the one image
    oneBatch,_ = imgDataBunch.one_item(tensorImg)
    oneBatch_im = vision.Image(imgDataBunch.denorm(oneBatch)[0])
    # convert batch tensor image to grayscale image with opencv
    cvIm = cv2.cvtColor(image2np(oneBatch_im.data), cv2.COLOR_RGB2GRAY)
    # attach hooks
    hook_a,hook_g = hooked_backward(m, oneBatch, cl)
    # get convolutional activations and average from channels
    acts = hook_a.stored[0].cpu()
    #avg_acts = acts.mean(0)

    # Grad-CAM
    grad = hook_g.stored[0][0].cpu()
    grad_chan = grad.mean(1).mean(1)
    grad.shape,grad_chan.shape
    mult = (acts*grad_chan[...,None,None]).mean(0)
    return mult, cvIm

# Then, modify our plotting func a bit
def plot_heatmap_overview(interp:ClassificationInterpretation, learner, imgDataBunch, classes=['Cargo', 'Carrier', 'Cruise', 'Military', 'Tankers']):
    # top losses will return all validation losses and indexes sorted by the largest first
    tl_val,tl_idx = interp.top_losses()
    #classes = interp.data.classes
    fig, ax = plt.subplots(3,4, figsize=(16,12))
    fig.suptitle('Grad-CAM\nPredicted / Actual / Loss / Probability',fontsize=20)
    # Random
    for i in range(4):
        random_index = random.randint(0,len(tl_idx))
        idx = tl_idx[random_index]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[0,i].imshow(im)
        ax[0,i].imshow(im, cmap=plt.cm.gray)
        ax[0,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[0,i].set_xticks([])
        ax[0,i].set_yticks([])
        ax[0,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[0,0].set_ylabel('Random samples', fontsize=16, rotation=0, labelpad=80)
    # Most incorrect or top losses
    for i in range(4):
        idx = tl_idx[i]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[1,i].imshow(im)
        ax[1,i].imshow(im, cmap=plt.cm.gray)
        ax[1,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[1,i].set_xticks([])
        ax[1,i].set_yticks([])
        ax[1,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[1,0].set_ylabel('Most incorrect\nsamples', fontsize=16, rotation=0, labelpad=80)
    # Most correct or least losses
    for i in range(4):
        idx = tl_idx[len(tl_idx) - i - 1]
        act, im = getHeatmap(idx, learner, imgDataBunch)
        H,W = im.shape
        _,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        cl = int(cl)
        ax[2,i].imshow(im)
        ax[2,i].imshow(im, cmap=plt.cm.gray)
        ax[2,i].imshow(act, alpha=0.5, extent=(0,H,W,0),
              interpolation='bilinear', cmap='inferno')
        ax[2,i].set_xticks([])
        ax[2,i].set_yticks([])
        ax[2,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[2,0].set_ylabel('Most correct\nsamples', fontsize=16, rotation=0, labelpad=80)

In [ ]:
plot_heatmap_overview(interp, learn1, data1)

In [82]:
fast_i2c = {v:i for i, v in learn1.data.train_ds.y.c2i.items()}

In [83]:
fast_i2c

{0: 'Cargo', 1: 'Carrier', 2: 'Cruise', 3: 'Military', 4: 'Tankers'}

In [ ]:
val_preds_tta = learn1.TTA(ds_type=DatasetType.Valid)

In [84]:
accuracy_score(y_pred=val_preds_tta[0].argmax(1).numpy(), y_true=val_preds_tta[1].numpy())

0.9592326139088729

In [85]:
f1_score(y_pred=val_preds_tta[0].argmax(1).numpy(), y_true=val_preds_tta[1].numpy(), average='weighted')

0.9591674192405736

In [ ]:
val_preds = learn1.get_preds()

In [86]:
accuracy_score(y_pred=val_preds[0].argmax(1).numpy(), y_true=val_preds[1].numpy())

0.9496402877697842

In [87]:
f1_score(y_pred=val_preds[0].argmax(1).numpy(), y_true=val_preds[1].numpy(), average='weighted')

0.9498235629492824

In [88]:
[Path(i).name for i in learn1.data.valid_ds.x.items]

['2835355.jpg',
 '2491051.jpg',
 '2862151.jpg',
 '2888474.jpg',
 '697630.jpg',
 '2543936.jpg',
 '2791673.jpg',
 '2702317.jpg',
 '2821170.jpg',
 '2751844.jpg',
 '2796480.jpg',
 '2885047.jpg',
 '2909296.jpg',
 '1334846.jpg',
 '2217944.jpg',
 '2492386.jpg',
 '2798524.jpg',
 '2662125.jpg',
 '2909044.jpg',
 '2824870.jpg',
 '2888390.jpg',
 '2806683.jpg',
 '2012371.jpg',
 '2870069.jpg',
 '2837657.jpg',
 '2838584.jpg',
 '625127.jpg',
 '2854009.jpg',
 '2903461.jpg',
 '2843767.jpg',
 '2895730.jpg',
 '1721123.jpg',
 '2900643.jpg',
 '1641595.jpg',
 '2861611.jpg',
 '2888400.jpg',
 '2891623.jpg',
 '2786171.jpg',
 '2903497.jpg',
 '2902689.jpg',
 '2872126.jpg',
 '2879438.jpg',
 '2796338.jpg',
 '2841730.jpg',
 '2876164.jpg',
 '1886879.jpg',
 '2827211.jpg',
 '2837655.jpg',
 '2806766.jpg',
 '2890619.jpg',
 '1188084.jpg',
 '1400705.jpg',
 '2798386.jpg',
 '2825277.jpg',
 '2770991.jpg',
 '2671011.jpg',
 '2849132.jpg',
 '2885147.jpg',
 '2823082.jpg',
 '2902572.jpg',
 '2902778.jpg',
 '2904657.jpg',
 '1949407.

In [89]:
test_preds_tta = learn1.TTA(ds_type=DatasetType.Test)

In [90]:
test_preds = learn1.get_preds(ds_type=DatasetType.Test)

In [91]:
test_img_names = [Path(i).name for i in learn1.data.test_ds.x.items]

In [101]:
test_pred_tta_df = pd.DataFrame({'image': test_img_names})
test_pred_tta_df['category'] = test_preds_tta[0].argmax(1).numpy()

In [102]:
test_pred_df = pd.DataFrame({'image': test_img_names})
test_pred_df['category'] = test_preds[0].argmax(1).numpy()

In [103]:
test_pred_tta_df['category'] = test_pred_tta_df['category'].map(fast_i2c)
test_pred_tta_df['category'] = test_pred_tta_df['category'].map(label_mapper_c2i).astype(int) + 1

test_pred_df['category'] = test_pred_df['category'].map(fast_i2c)
test_pred_df['category'] = test_pred_df['category'].map(label_mapper_c2i).astype(int) + 1

In [104]:
test_pred_tta_df.to_csv('resnet50_tta.csv', index=False)
test_pred_df.to_csv('resnet50.csv', index=False)

In [105]:
# LB 0.951670584793539
FileLink('resnet50.csv')

/kaggle/working/resnet50.csv

In [106]:
# LB 0.956144193630911
FileLink('resnet50_tta.csv')

/kaggle/working/resnet50_tta.csv

In [ ]:
Path('../submissions/resnet50/').mkdir(exist_ok=True)